In [175]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np



########################
### Global variables ###
########################

# Size of the agent's pool
AGENT_WATER_VOLUME_MAX = 300
# Price of missing a 1 water unit of consumer: 
# e.g agent's water volume = 10, consumer demand = 20, price = (20-10) * PENALTY_PER_WATER_UNIT
PENALTY_PER_WATER_UNIT = 1000

TOTAL_DAILY_DEMAND = 1000



#########################
### Price calculation ###
#########################

def hourly_demand_means():
    # Hourly water demand according to: 
    hourly_demand = np.array([
        2, 2, 2, 2, 3, 5, 10, 12, 10, 8, 6, 5, 5, 5, 5, 6, 7, 9, 10, 9, 6, 4, 3, 2
    ])
    # Normalize: Daily demand is 1000
    hourly_demand = (hourly_demand / hourly_demand.sum()) * TOTAL_DAILY_DEMAND

    # on day 7: less demand
    hourly_demand = np.append(np.tile(hourly_demand,6), hourly_demand / 2)
    return hourly_demand



def sample_demand(hour, std=10):
    # Sample from a normal distribution, ensure demand is non-negative
    hourly_demand = hourly_demand_means()
    mean_demand = hourly_demand[hour] # Mean demand follows a daily pattern, higher during the day (6 AM to 6 PM)
    return max(0, np.random.normal(mean_demand, std)) # When std_dev is relatively low we will get lines that are very very close to the original function.


def calculate_price(amount_to_buy: float, base_price: float) -> float:
    return amount_to_buy * base_price + 0.05 * base_price * (amount_to_buy -1) ** 2


def get_water_prices():
    # expensive hours: 8:00-16:00
    source_1_price = np.ones(24)
    source_1_price[8:16] = 2
    source_1_price = np.tile(source_1_price,7)
    return source_1_price


##############################################
### Environment Definition : WaterSupplyEnv ###
##############################################

class WaterSupplyEnv(gym.Env):

    def __init__(self, max_weeks=10):
        super().__init__()
        # State: [water_level, price_A, price_B, demand, current_hour]
        self.max_weeks = max_weeks
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(5,), dtype=np.float32)

        # Actions: [buy_from_A, buy_from_B]
        self.action_space = spaces.Box(low=0, high=AGENT_WATER_VOLUME_MAX, shape=(2,), dtype=np.float32)

        # Environment parameters
        self.max_water_level = AGENT_WATER_VOLUME_MAX
        self.source_A_base_prices = get_water_prices()
        
        # Source B is more expensive
        self.source_B_base_prices = 1.5 * self.source_A_base_prices

    def _get_obs(self):
        return np.array([self.water_level,
                         self.price_A,
                         self.price_B,
                         self.demand,
                         self.current_hour])
    def _get_info(self):
        # return more expicit info, should be used only for debugging
        return {"water_level": self.water_level,
                "price_A": self.price_A,
                "price_B": self.price_B,
                "demand": self.demand,
                "current_hour": self.current_hour}

    def reset(self, seed = None, options = None):
        # seed self.np_random
        super().reset(seed=seed)

        # initialize env state
        self.total_hours = 0
        self.current_hour = 0
        self.water_level = self.max_water_level # Initial water level
        self.demand = sample_demand(self.current_hour) # Initial demand
        self.price_A = self.source_A_base_prices[self.current_hour]
        self.price_B = self.source_B_base_prices[self.current_hour]
        self.total_reward = 0
        return self._get_obs(), self._get_info() # obs, info

    def step(self, action):
        buy_from_A, buy_from_B = action

        # Calculate penalty for unmet demand
        unmet_demand_penalty = max(0, (self.demand - self.water_level) * PENALTY_PER_WATER_UNIT)
        self.water_level = max(0, self.water_level - self.demand)

        
        # Costs and revenues
        cost_A = calculate_price(buy_from_A, self.source_A_base_prices[self.current_hour])
        cost_B = calculate_price(buy_from_B, self.source_B_base_prices[self.current_hour])
        # Update water stock
        self.water_level += buy_from_A + buy_from_B

        # Calculate reward
        reward = (- cost_A - cost_B) - unmet_demand_penalty # reward function
        self.total_reward += reward

        # Ensure constraints
        self.water_level = min(self.max_water_level, self.water_level)  # Ensure water level does not surpass maximum

        # Update state
        self.total_hours += 1
        self.current_hour = (self.current_hour + 1) % 168
        self.demand = sample_demand(self.current_hour)
        self.price_A = self.source_A_base_prices[self.current_hour]
        self.price_B = self.source_B_base_prices[self.current_hour]

        observation = self._get_obs()

        # terminated = False  # TODO: Define terminal conditions if necessary
        terminated = True if (self.max_weeks and self.total_hours >= 168 * self.max_weeks) else False
        truncated = False # TODO: Define truncated conditions if necessary
        info = self._get_info()
        return observation, reward, terminated, truncated, info
    
    def seed(self, seed=None):
        np.random.seed(seed)

    def render(self, mode="human", **kwargs):
        if kwargs.get("close", False):
            # If 'close' is passed, do nothing
            return
        print(f"Current_Hour: {self.current_hour}, Water Stock: {self.water_level}, Price A: {self.price_A}, Price B: {self.price_B}, Demand: {self.demand}, MONEY: {self.total_reward}")



In [176]:
# # Demo env use
# env = WaterSupplyEnv(max_weeks=10)
# env.reset()
# i = 0
# for _ in range(999999):
#     i += 1
#     print("Step: ", i)
#     # action = env.action_space.sample()
#     action = [10, 10]
#     print("Chosen action: ", action)
#     observation, reward, terminated, truncated, info = env.step(action)
#     print("Render: ")
#     env.render()
#     if terminated:
#         break


 # Discretisize Env

In [177]:
class DiscreteActions(gym.ActionWrapper):
    # size_of_purchase: the resolution of water that can be bought: 10 => can by in 
    # units of 10, 20, 30 ...
    def __init__(self, env, max_water_volume, size_of_purchase):
        super().__init__(env)
        # how many discrete actions per source (sqrt of the size of the discrete action space)
        self.action_amount = max_water_volume // size_of_purchase + 1
        # DQN need dicrete
        self.action_space = spaces.Discrete(self.action_amount * self.action_amount)
    
    def action_to_quantity(self, action):
            from_source_1 = action // self.action_amount
            from_source_2 = action % self.action_amount
            return [from_source_1, from_source_2]

    def action(self, action):
        # action is of discrete space with size: action_amount * action_amount
        from_source_1, from_source_2 = self.action_to_quantity(action)
        return [from_source_1, from_source_2]
    
# Wrap the environment
env = WaterSupplyEnv(max_weeks=3)
env = DiscreteActions(env, size_of_purchase=50, max_water_volume=300)

# Now, env.action_space is Discrete, so DQN will work
print("Modified action space:", env.action_space)


Modified action space: Discrete(49)


# Stable Baselines

In [178]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

In [179]:
from stable_baselines3 import DQN

In [180]:
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.001)
model.learn(total_timesteps=10000, log_interval=4)
model.save("dqn_water1")
del model # remove to demonstrate saving and loading
model = DQN.load("dqn_water1")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 504       |
|    ep_rew_mean      | -1.59e+07 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 4         |
|    fps              | 1181      |
|    time_elapsed     | 1         |
|    total_timesteps  | 2016      |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss             | 1.98e+04  |
|    n_updates        | 478       |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 504       |
|    ep_rew_mean      | -1.56e+07 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 8         |
|    fps              | 1132      |
|    time_elapsed     | 3         |
|    total_timesteps  | 4032      |
| trai

In [184]:
obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print("Reward:", reward)
    env.render()
    if terminated or truncated:
        break
        obs, info = env.reset()

Reward: -4.075
Current_Hour: 1, Water Stock: 285.54992750619476, Price A: 1.0, Price B: 1.5, Demand: 0, MONEY: -4.075
Reward: -4.075
Current_Hour: 2, Water Stock: 288.54992750619476, Price A: 1.0, Price B: 1.5, Demand: 13.296241776089794, MONEY: -8.15
Reward: -4.075
Current_Hour: 3, Water Stock: 278.25368573010496, Price A: 1.0, Price B: 1.5, Demand: 17.11699481458627, MONEY: -12.225000000000001
Reward: -4.075
Current_Hour: 4, Water Stock: 264.13669091551867, Price A: 1.0, Price B: 1.5, Demand: 4.896654923429658, MONEY: -16.3
Reward: -4.075
Current_Hour: 5, Water Stock: 262.240035992089, Price A: 1.0, Price B: 1.5, Demand: 42.9560299346566, MONEY: -20.375
Reward: -4.075
Current_Hour: 6, Water Stock: 222.2840060574324, Price A: 1.0, Price B: 1.5, Demand: 65.89500267073947, MONEY: -24.45
Reward: -4.075
Current_Hour: 7, Water Stock: 159.38900338669293, Price A: 1.0, Price B: 1.5, Demand: 94.59665926441285, MONEY: -28.525
Reward: -4.075
Current_Hour: 8, Water Stock: 67.79234412228008, Pric